# MATH 441 Learning Portfolio 1 

## September 23, 2022

In the first two weeks of course, I have learned how to solve a linear optimization problem by using python to transfer the given data to a certain form, and then take the pivot operations to approach the optimal solution. 

In the first portfolio, I will solve a linear programming problem.
- 1. Solving the problem by using pivot operation. 
- 2. Checking the result by SciPy Solver

In [2]:
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt

$$
\begin{array}{rc}
\text{Maximize:} & z = 3x_1 + 2x_2 + 4x_3  \\
\text{Subject to:} & x_1 + x_2 + 2x_3  \leqslant  4 \\
& 2x_1\qquad + 3x_3 \leqslant 5 \\
& 2x_1 + x_2 + 3x_3 \leqslant 7 \\
& x_1 \geq 0, x_2 \geq 0, x_3\geq 0
\end{array}
$$


Present the problem in matrix


$$
A = \left[ \begin{array}{rrr} 1 & 1 & 2 \\ 2 & 0 & 3 \\ 2 & 1 & 3 \end{array} \right]
\hspace{1in}
\mathbf{b} = \left[ \begin{array}{r} 4 \\ 5 \\ 7\end{array} \right]
\hspace{1in}
\mathbf{c} = \left[ \begin{array}{r} 3 \\ 2 \\ 4 \end{array} \right]
$$


**Step 1: Combining the matrix together in the T form**

$$
T = 
\begin{bmatrix}
A & I & \mathbf{b} \\
\mathbf{c}^T & 0 & 0
\end{bmatrix}
$$


In [15]:
A = np.array([[1.,1.,2.],[2.,0.,3.],[2.,1.,3.]])
m,n = A.shape
I = np.eye(m)
c = np.array([3.,2.,4.])
b = np.array([4.,5.,7.])
T = np.vstack([ np.hstack([A,I,b.reshape((m,1))]) , np.hstack([c,np.zeros(m+1)]) ])
print(T)

[[1. 1. 2. 1. 0. 0. 4.]
 [2. 0. 3. 0. 1. 0. 5.]
 [2. 1. 3. 0. 0. 1. 7.]
 [3. 2. 4. 0. 0. 0. 0.]]


**Step 2: Taking the pivot operation by using function pivot(T, enter, leave)** 
<br>
**T**: the result matrix by step 1
<br>
**enter**: the column of entering variable, which is chosen the most positive coeffcient of non-basic variable from the last row of T, $\mathbf{c}^T$.
<br>
**leave**: the row of leaving variable.

In [16]:
def pivot(T,enter,leave):
    E = np.eye(T.shape[0])
    E[:,leave] = -T[:,enter]/T[leave,enter]
    E[leave,leave] = 1/T[leave,enter]
    return E@T

### How to determine which row to leave ?

In our example, we enter the column 2 from T. The other nonbasic variable in column 0 and 1 would be 0.
<br>
we now have

$$
T = \left[ \begin{array}{rrr} 0 & 0 & \color{blue}{2}  & 1 & 0 & 0 & \color{red}{4} \\ 0 & 0 & \color{blue}{3} & 0 & 1 & 0 & \color{red}{5} \\ 0 & 0 & \color{blue}{3} & 0 & 0 & 1 & \color{red}{7} \\ 3 & 2 & 4 & 0 & 0 & 0 & 0\\ \end{array} \right]
$$

To get the least upper bound of basic variable, we use number in the last column (noted red) divide the enter column (noted blue), then pick the least result row to be the leaving row.

In [24]:
T1 = pivot(T, 2, 1)
print(T1.round(2))

[[-0.33  1.    0.    1.   -0.67  0.    0.67]
 [ 0.67  0.    1.    0.    0.33  0.    1.67]
 [ 0.    1.    0.    0.   -1.    1.    2.  ]
 [ 0.33  2.    0.    0.   -1.33  0.   -6.67]]


In [25]:
T2 = pivot(T1, 1, 0)
print(T2.round(2))

[[-0.33  1.    0.    1.   -0.67  0.    0.67]
 [ 0.67  0.    1.    0.    0.33  0.    1.67]
 [ 0.33  0.    0.   -1.   -0.33  1.    1.33]
 [ 1.    0.    0.   -2.    0.    0.   -8.  ]]


In [26]:
T3 = pivot(T2, 0, 1)
print(T3.round(2))

[[  0.    1.    0.5   1.   -0.5   0.    1.5]
 [  1.    0.    1.5   0.    0.5   0.    2.5]
 [  0.    0.   -0.5  -1.   -0.5   1.    0.5]
 [  0.    0.   -1.5  -2.   -0.5   0.  -10.5]]


### Checking by using SciPy Solver 

In [27]:
from scipy.optimize import linprog

In [28]:
A = np.array([[1.,1.,2.],[2.,0.,3.],[2.,1.,3.]])
c = np.array([3.,2.,4.])
b = np.array([4.,5.,7.]) 
linprog(-c,A,b)

           con: array([], dtype=float64)
 crossover_nit: 0
         eqlin:  marginals: array([], dtype=float64)
  residual: array([], dtype=float64)
           fun: -10.5
       ineqlin:  marginals: array([-2. , -0.5, -0. ])
  residual: array([0. , 0. , 0.5])
         lower:  marginals: array([0. , 0. , 1.5])
  residual: array([2.5, 1.5, 0. ])
       message: 'Optimization terminated successfully. (HiGHS Status 7: Optimal)'
           nit: 3
         slack: array([0. , 0. , 0.5])
        status: 0
       success: True
         upper:  marginals: array([0., 0., 0.])
  residual: array([inf, inf, inf])
             x: array([2.5, 1.5, 0. ])

By checking the 